<a href="https://colab.research.google.com/github/krMaynard/genai/blob/main/gemini_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from IPython.display import display, Markdown
from google.colab import userdata
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64
import random

# Generate a random integer between 0 and 100 (inclusive)
random_number = random.randint(0, 100)

secret_api_key = userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=secret_api_key)

contents = '''
Generate a hyperrealistic, cinematic photograph of a surreal moment in a modern Seoul subway station. A full-grown, elegant giraffe is sitting upright on a polished metal bench, impeccably dressed in a tailored dark grey business suit with a crisp white shirt and a silk tie. Next to the giraffe sits a Korean teenage boy in his late teens, wearing contemporary streetwear (a stylish hoodie, jeans, and sneakers). The boy is turned towards the giraffe, talking to it with an earnest and animated expression, gesturing with one hand. The giraffe appears to be listening intently.

The subway station is clean and brightly lit with a mix of cool fluorescent overheads and warm accent lights. In the background, digital signs display train information in glowing Hangul (Korean script). The tiled walls and floor are pristine, reflecting the lights. The yellow safety line is visible on the edge of the platform.

Capture the scene with a shallow depth of field, keeping the giraffe and the boy in sharp focus while the background is softly blurred. The lighting should create soft shadows, enhancing the realism and the quiet, intimate nature of their bizarre conversation.

**Style:** Photorealistic, 8K resolution, ultra-detailed, sharp focus, shot on a Sony A7R IV with a 50mm f/1.2 lens.
'''

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

image_filename = f'gemini-native-image_{random_number}.png'

for part in response.candidates[0].content.parts:
  if part.text is not None:
    display(Markdown(part.text))
  elif part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save(image_filename)
    print(f'Image saved to {image_filename}')
    image.show()


I will generate a photorealistic image in 8K resolution depicting a surreal scene in a modern Seoul subway station. A sophisticated giraffe in a dark grey business suit, white shirt, and silk tie sits attentively on a metal bench next to a Korean teenage boy in streetwear. The boy is animatedly talking to the giraffe, who seems to be listening. The clean, bright subway station has digital signs in Hangul, tiled surfaces, and a yellow safety line. The foreground with the giraffe and boy will be sharply focused with a shallow depth of field, and the background will be softly blurred. Soft shadows will enhance the realism.



Image saved to gemini-native-image_26.png
